# Demo
Examples of how to use this repo

These are the commands for VPT with `p=50`:

VPT-deep
```bash
    --config-file configs/prompt/*.yaml
    MODEL.TRANSFER_TYPE "prompt" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.NUM_TOKENS "50" \
    MODEL.PROMPT.DROPOUT "0.0" 
```
   
VPT-shallow (we don't use dropout for VPT-shallow)
```bash
    --config-file configs/prompt/*.yaml
    MODEL.TRANSFER_TYPE "prompt" \
    MODEL.PROMPT.DEEP "False" \
    MODEL.PROMPT.NUM_TOKENS "50" \
    MODEL.PROMPT.DROPOUT "0.0" 
```

Other transfer protocols presented in the paper:

Full
```bash
    --config-file configs/finetune/*.yaml
```

Head-oriented methods:

- Linear:
```bash
    --config-file configs/linear/*.yaml
```

- MLP-3 (3 layer MLP):
```bash
    --config-file configs/linear/*.yaml \
    MODEL.MLP_NUM "2"
```

- Partial-1:
```bash
    --config-file configs/finetune/*.yaml \
    MODEL.TRANSFER_TYPE "partial-1"
```


Backbone-oriented methods:

- Sidetune:
```bash
    --config-file configs/linear/*.yaml
    MODEL.TRANSFER_TYPE  "side" 
```

- Bias: 
```bash
    --config-file configs/finetune/*.yaml \
    MODEL.TRANSFER_TYPE "tinytl-bias"
```

- Adapters with `r=128`:
```bash
    --config-file configs/finetune/*.yaml
    MODEL.ADAPTER.REDUCATION_FACTOR "128"
    MODEL.TRANSFER_TYPE "adapter" 
```

##  train.py
The main script is `train.py`. Note for VTAB data, this script handles the final runs with 1000 training data. See `tune_vtab.py` for the full tune + final runs settings. Here are some examples.

Note: it's recommended to directly use terminal for these command.

In [1]:
import torch

/home/yourenz/.conda/envs/prompt/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%%bash

source activate prompt

model_root=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/pretrained/
data_path=/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/datasets/cub
output_dir=/home/yourenz/SynergisticEfficiency/exps

python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitti16_imagenet21k" \
        SEED 42 \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}" 

In [ ]:
%%bash
# launch final training with five random seeds for VTAB-dmlab, sun397 and eurosat. The hyperparameters are the same from our paper.
model_root=<MODEL_ROOT>
data_path=<DATA_PATH>
output_dir=<OUTPUT_DIR>
        
# vtab-structured: dmlab
# base_lr = 1.0
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-dmlab" \
        DATA.NUMBER_CLASSES "6" \
        SOLVER.BASE_LR "0.25" \
        SOLVER.WEIGHT_DECAY "0.001" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

# vtab-natural: sun397
# base_lr = 25
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "128" \
        MODEL.PROMPT.NUM_TOKENS "5" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-sun397" \
        DATA.NUMBER_CLASSES "397" \
        SOLVER.BASE_LR "12.5" \
        SOLVER.WEIGHT_DECAY "0.0001" \
        SOLVER.TOTAL_EPOCH "100" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

# vtab-specialized: vtab-eurosat
# base_lr = 1
# lr = base_lr / 256 * cfg.DATA.BATCH_SIZE
for seed in "42" "44" "82" "100" "800"; do
    python train.py \
        --config-file configs/prompt/cub.yaml \
        MODEL.TYPE "vit" \
        DATA.BATCH_SIZE "64" \
        MODEL.PROMPT.NUM_TOKENS "100" \
        MODEL.PROMPT.DEEP "True" \
        MODEL.PROMPT.DROPOUT "0.1" \
        DATA.FEATURE "sup_vitb16_imagenet21k" \
        DATA.NAME "vtab-eurosat" \
        DATA.NUMBER_CLASSES "10" \
        SOLVER.BASE_LR "0.25" \
        SOLVER.WEIGHT_DECAY "0.001" \
        SOLVER.TOTAL_EPOCH "100" \
        SEED ${seed} \
        MODEL.MODEL_ROOT "${model_root}" \
        DATA.DATAPATH "${data_path}" \
        OUTPUT_DIR "${output_dir}/seed${seed}" 
done

## Get results

In [1]:
import glob
import pandas as pd

from src.utils.vis_utils import get_df, average_df

LOG_NAME = "logs.txt"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
root = <MODEL_ROOT>
df_list=[]
for seed in ["42", "44", "82", "100", "800"]:
#     model_type = f"adapter_{r}"
    files = glob.glob(f"{root}/seed{seed}/*/sup_vitb16_imagenet21k/*/*/{LOG_NAME}")
    for f in files:
        df = get_df(files, f"seed{seed}", root, is_best=False, is_last=True)
        if df is None:
            continue
        df["seed"] = seed
    df_list.append(df)

df= pd.concat(df_list)
df["type"] = "VPT"
df

seed800: 100%|██████████| 3/3 [00:00<00:00, 252.79it/s]


,data,feature,lr,wd,total_params,tuned_params,tuned / total (%),batch_size,l-val_top1,l-test_top1,best_epoch,file,total_time,seed,type
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,46.88,76 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:03,42,VPT
0,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.00,38 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:57,42,VPT
1,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,52.57,14 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:57,42,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,99.5,46.25,85 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:09,44,VPT
0,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.54,41 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:38,44,VPT
1,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,49.03,32 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:53,44,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,46.14,65 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:04:18,82,VPT
1,vtab-eurosat,sup_vitb16_imagenet21k,1.0,0.0010,86727946,929290,1.0715,64,100.0,96.67,41 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:42,82,VPT
0,vtab-sun397,sup_vitb16_imagenet21k,25.0,0.0001,86150029,351373,0.4079,128,100.0,52.45,8 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 00:44:51,82,VPT
2,vtab-dmlab,sup_vitb16_imagenet21k,1.0,0.0010,86724870,926214,1.0680,64,100.0,47.41,76 | 100,/fsx/menglin/experiments/2022prompt/output/rel...,0 days 01:06:15,100,VPT


Take average of 5 runs for each dataset:

In [4]:
# LR represents the base learning rate, not the scaled one.
f_df = average_df(df, metric_names=["l-test_top1"], take_average=True)
f_df

,data,feature,type,total_runs,l-test_top1,l-test_top1-std,lr,wd,total_params,tuned_params,tuned / total (%),batch_size,l-val_top1,total_time,seed
0,vtab-dmlab,sup_vitb16_imagenet21k,VPT,5,46.62,0.47,1.0,0.0010,86724870,926214,1.0680,64,100.0,0 days 01:06:03,42
2,vtab-eurosat,sup_vitb16_imagenet21k,VPT,5,96.15,0.45,1.0,0.0010,86727946,929290,1.0715,64,100.0,0 days 00:44:57,42
1,vtab-sun397,sup_vitb16_imagenet21k,VPT,5,51.57,1.40,25.0,0.0001,86150029,351373,0.4079,128,100.0,0 days 00:44:57,42


## tune*.py
Tune vtab or fgvc datasets. 

In [ ]:
%%bash
# Tune VTAB-caltech101 with VPT:
python tune_vtab.py \
    --train-type "prompt" \
    --config-file configs/prompt/cub.yaml \
    MODEL.TYPE "vit" \
    DATA.BATCH_SIZE "128" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.DROPOUT "0.1" \
    MODEL.PROMPT.NUM_TOKENS "10" \
    DATA.FEATURE "sup_vitb16_imagenet21k" \
    DATA.NAME "vtab-caltech101" \
    DATA.NUMBER_CLASSES "102" \
    DATA.DATAPATH <DATA_PATH> \
    MODEL.MODEL_ROOT <MODEL_ROOT> \
    OUTPUT_DIR <OUTPUT_PATH> 

# Tune CUB with VPT:
python tune_fgvc.py \
    --train-type "prompt" \
    --config-file configs/prompt/cub.yaml \
    MODEL.TYPE "vit" \
    DATA.BATCH_SIZE "128" \
    MODEL.PROMPT.DEEP "True" \
    MODEL.PROMPT.DROPOUT "0.1" \
    MODEL.PROMPT.NUM_TOKENS "10" \
    DATA.FEATURE "sup_vitb16_imagenet21k" \
    DATA.DATAPATH <DATA_PATH> \
    MODEL.MODEL_ROOT <MODEL_ROOT> \
    OUTPUT_DIR <OUTPUT_PATH> 

## Backbone choices

- Swin-B

```bash
    MODEL.TYPE "swin" \
    DATA.FEATURE "swinb_imagenet22k_224"
```

- ResNet-50 (VPT with prompt location == pad)

```bash
    MODEL.TYPE "resnet" \
    DATA.FEATURE "imagenet_sup_rn50" \
    SOLVER.OPTIMIZER "sgd" \
    MODEL.PROMPT.LOCATION "pad" \
    MODEL.PROMPT.NUM_TOKENS "5" 
```

- ConvNeXt-Base (VPT with prompt location == pad)

```bash
    MODEL.TYPE "resnext" \
    DATA.FEATURE "imagenet22k_sup_rnx_base" \
    MODEL.PROMPT.LOCATION "pad" \
    MODEL.PROMPT.NUM_TOKENS "5" 
```

ViT with self-supervised pre-training objectives:
    
- MAE

```bash
MODEL.TYPE "ssl-vit" \
DATA.FEATURE "mae_vitb16"
```

- MoCo-v3

```bash
MODEL.TYPE "ssl-vit" \
DATA.FEATURE "mocov3_vitb" 
```

## ViT (VPT)

In [2]:
from src.configs.config import get_cfg
from src.models.build_model import build_model
import json
model_root='/gpfs/accounts/eecs553w23_class_root/eecs553w23_class/shared_data/yourenz_group/pretrained/'

In [5]:
cfg_file = './configs/prompt/cub.yaml'
cfg = get_cfg()
cfg.merge_from_file(cfg_file)
cfg.MODEL.MODEL_ROOT = model_root
cfg.DATA.FEATURE = 'sup_vitb16_imagenet21k'
print(cfg.dump())

CUDNN_BENCHMARK: false
DATA:
  BATCH_SIZE: 128
  CLASS_WEIGHTS_TYPE: none
  CROPSIZE: 224
  DATAPATH: ''
  FEATURE: sup_vitb16_imagenet21k
  MULTILABEL: false
  NAME: CUB
  NO_TEST: false
  NUMBER_CLASSES: 200
  NUM_WORKERS: 4
  PERCENTAGE: 1.0
  PIN_MEMORY: true
DBG: false
DIST_BACKEND: nccl
DIST_INIT_FILE: ''
DIST_INIT_PATH: env://
MODEL:
  ADAPTER:
    REDUCATION_FACTOR: 8
    STYLE: Pfeiffer
  LINEAR:
    DROPOUT: 0.1
    MLP_SIZES: []
  MLP_NUM: 0
  MODEL_ROOT: /gpfs/accounts/eecs553w23_class_root/eecs553w23_class/shared_data/yourenz_group/pretrained/
  PROMPT:
    CLSEMB_FOLDER: ''
    CLSEMB_PATH: ''
    DEEP: false
    DEEP_SHARED: false
    DROPOUT: 0.0
    FORWARD_DEEP_NOEXPAND: false
    INITIATION: random
    LOCATION: prepend
    NUM_DEEP_LAYERS: null
    NUM_TOKENS: 5
    PROJECT: -1
    REVERSE_DEEP: false
    SAVE_FOR_EACH_EPOCH: false
    VIT_POOL_TYPE: original
  SAVE_CKPT: false
  TRANSFER_TYPE: prompt
  TYPE: vit
  WEIGHT_PATH: ''
NUM_GPUS: 0
NUM_SHARDS: 1
OUTPUT_DI

In [3]:
import e_vit, timm

In [4]:
model_vpt, cur_device = build_model(cfg)
model_vpt

src.models.vit_models.ViT

In [5]:
import torch
input = torch.rand((4, 3, 224, 224))
output_vpt = model_vpt(input)
print(output_vpt.shape)

block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
block input:  torch.Size([4, 202, 768])
hidden states:  torch.Size([4, 202, 768])
torch.Size([4, 2

In [6]:
e_vit.patch.timm(model_vpt, [1,1,1,0.7,1,1,0.7,1,1,0.7,1,1])
model_vpt.__class__

e_vit.patch.timm.make_tome_class.<locals>.EvitVisionTransformer

In [7]:
import torch
input = torch.rand((4, 3, 224, 224))
# output_timm = model_timm(input)
# print(output_timm.shape)
output_vpt = model_vpt(input)
print(output_vpt.shape)

202
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1]} 1
202
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1]} 1
202
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1]} 1
202
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [1, 1, 0.7, 1, 1, 0.7, 1, 1]} 0.7
143
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [1, 0.7, 1, 1, 0.7, 1, 1]} 1
143
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [0.7, 1, 1, 0.7, 1, 1]} 1
143
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [1, 1, 0.7, 1, 1]} 0.7
102
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [1, 0.7, 1, 1]} 1
102
{'keep_rate': [1, 1, 1, 0.7, 1, 1, 0.7, 1, 1, 0.7, 1, 1], 'fuse': True, 'r': [0.7, 1, 1]} 1
102
{'

In [8]:
import inspect
for i in inspect.getmembers(model_vpt):
     
    # to remove private and protected
    # functions
    if not i[0].startswith('_'):
         
        # To remove other methods that
        # doesnot start with a underscore
        if not inspect.ismethod(i[1]):
            print(i)

('T_destination', ~T_destination)
('cfg', CfgNode({'DBG': False, 'OUTPUT_DIR': '', 'RUN_N_TIMES': 1, 'CUDNN_BENCHMARK': False, 'NUM_GPUS': 0, 'NUM_SHARDS': 1, 'SEED': None, 'MODEL': CfgNode({'TRANSFER_TYPE': 'prompt', 'WEIGHT_PATH': '', 'SAVE_CKPT': False, 'MODEL_ROOT': '/gpfs/accounts/eecs598s010w23_class_root/eecs598s010w23_class/shared_data/yourenz_group/pretrained/', 'TYPE': 'vit', 'MLP_NUM': 0, 'LINEAR': CfgNode({'MLP_SIZES': [], 'DROPOUT': 0.1}), 'PROMPT': CfgNode({'NUM_TOKENS': 5, 'LOCATION': 'prepend', 'INITIATION': 'random', 'CLSEMB_FOLDER': '', 'CLSEMB_PATH': '', 'PROJECT': -1, 'DEEP': False, 'NUM_DEEP_LAYERS': None, 'REVERSE_DEEP': False, 'DEEP_SHARED': False, 'FORWARD_DEEP_NOEXPAND': False, 'VIT_POOL_TYPE': 'original', 'DROPOUT': 0.0, 'SAVE_FOR_EACH_EPOCH': False}), 'ADAPTER': CfgNode({'REDUCATION_FACTOR': 8, 'STYLE': 'Pfeiffer'})}), 'SOLVER': CfgNode({'LOSS': 'softmax', 'LOSS_ALPHA': 0.01, 'OPTIMIZER': 'sgd', 'MOMENTUM': 0.9, 'WEIGHT_DECAY': 0.01, 'WEIGHT_DECAY_BIAS': 0, '

In [14]:
for i in inspect.getmembers(model_timm):
     
    # to remove private and protected
    # functions
    if not i[0].startswith('_'):
         
        # To remove other methods that
        # doesnot start with a underscore
        if not inspect.ismethod(i[1]):
            print(i)

('T_destination', ~T_destination)
('blocks', Sequential(
  (0): Block(
    (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (attn): Attention(
      (qkv): Linear(in_features=768, out_features=2304, bias=True)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (proj): Linear(in_features=768, out_features=768, bias=True)
      (proj_drop): Dropout(p=0.0, inplace=False)
    )
    (drop_path): Identity()
    (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (mlp): Mlp(
      (fc1): Linear(in_features=768, out_features=3072, bias=True)
      (act): GELU()
      (fc2): Linear(in_features=3072, out_features=768, bias=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
  )
  (1): Block(
    (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
    (attn): Attention(
      (qkv): Linear(in_features=768, out_features=2304, bias=True)
      (attn_drop): Dropout(p=0.0, inplace=False)
      (proj): Linear(in_features=768, out_features=768, bias=T

In [20]:
import torch
import torch.nn as nn
class Attention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)
        # k: [b, num_heads, N, dim // num_heads]
        print(k.shape)
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [25]:
input = torch.rand((4, 10, 72))
attn = Attention(dim=72)

In [26]:
ouput = attn(input)
print(ouput.shape)

torch.Size([4, 8, 10, 9])
torch.Size([4, 10, 72])
